# GTI770 - TP2

In [38]:
%matplotlib inline   
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
import graphviz
import decimal
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras import optimizers as opt

from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import GridSearchCV

# from sklearn import tree
# from sklearn.naive_bayes import GaussianNB
# from sklearn.naive_bayes import MultinomialNB
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.ensemble import RandomForestClassifier
# from mdlp.discretization import MDLP
# from sklearn.preprocessing import MinMaxScaler

In [6]:
#Utility methods
def SplitVectorData_Holdout(primitives_vector, train_portion):
    
    size = len(primitives_vector)
    subsize = len(primitives_vector[0])
    
    val_portion = (1 - train_portion) / 2
    test_portion = (1 - train_portion) / 2

    nbTrain = int(size * train_portion)
    nbVal = int(size * val_portion)
    nbTest = int(size * test_portion)

    array_train = np.zeros((nbTrain, subsize), dtype=np.float64)
    array_val = np.zeros((nbVal, subsize), dtype=np.float64)
    array_test = np.zeros((nbTest, subsize), dtype=np.float64)

    array_train = primitives_vector[:nbTrain]
    array_val = primitives_vector[nbTrain : nbTrain + nbVal]
    array_test = primitives_vector[-nbTest:]
    return array_train, array_val, array_test

def SplitVectorData_KFold(primitives_vector, k, test_portion):
    
    size = len(primitives_vector)
    subsize = len(primitives_vector[0])
    subsetNb = int(size * (1 - test_portion) / k)
    testNb = int(size * test_portion)
    
    array_kfold_train = np.zeros((k, subsetNb, subsize), dtype=np.float64)
    array_kfold_test = np.zeros((testNb, subsize), dtype=np.float64)
    
    for i in range(0, k):
        if i == 0 :
            array_kfold_train[i] = primitives_vector[i * subsetNb]
        else :
            array_kfold_train[i] = primitives_vector[(i-1) * subsetNb : i * subsetNb]
    
    array_kfold_test = primitives_vector[-testNb:]

    return array_kfold_train, array_kfold_test

def concatenateWithoutTestPortion(full_array, index):
    result_array = []
    for i in range(0, len(full_array)):
        if len(result_array) == 0:
            result_array = full_array[i]
        elif i != index :
            result_array = np.concatenate((result_array, full_array[i]), axis=0)
    return result_array

def scores_mean(array_scores):
    
    size = len(array_scores)
    subsize = len(array_scores[0])
    array_mean = np.zeros(subsize, dtype=float)
    
    for i in range(0, size):
        for j in range(0, subsize):
            array_mean[j] += array_scores[i][j]
            
    for j in range(0, subsize):
            array_mean[j] = array_mean[j] / size
    
    return array_mean

def UseModelOnTestData(array, model, transformer=None):
    array_train, array_val, array_test = SplitVectorData_Holdout(array, 0.6)
    if transformer is not None:
        transform_train, transform_val, transform_test, Y_train, Y_val, Y_test = TransformData(array_train, array_val, array_test, len(array_train[0]), transformer)
    array_model_result, array_prediction_train_result, array_prediction_val_result, array_prediction_test_result, array_train_result, array_val_result, array_test_result = GenerateModelDataFromTransform(transform_train, transform_val, transform_test, Y_train, Y_val, Y_test, len(array[0]), model)
    result = [[array_test_result, array_prediction_test_result]]
    return GenerateScores(result) 

# ----- For debug -----
#Filter_train, Filter_val, Filter_test = SplitVectorDataTrainValTest(Filter, 0.6)
#print(len(Filter_train))
#print(len(Filter_val))
#print(len(Filter_test))
#print(len(Filter))
#----------------------

In [7]:
#Transforms and Models generation methods
def GenerateModelDataFromVector(array_train, array_val, array_test, num_features, chosen_model):
    
    num_features = num_features - 1
    
    data_train = array_train
    X_train  = data_train[:,0:num_features]
    Y_train  = data_train[:,num_features]
    
    data_val = array_val
    X_val = data_val[:,0:num_features]
    Y_val = data_val[:,num_features]
    
    data_test = array_test
    X_test = data_test[:,0:num_features]
    Y_test = data_test[:,num_features]
    
    
    model = chosen_model
    model = model.fit(X_train, Y_train)
    prediction_train = model.predict(X_train)
    prediction_val = model.predict(X_val)
    prediction_test = model.predict(X_test)
    return model, prediction_train, prediction_val, prediction_test, Y_train, Y_val, Y_test

def TransformData(array_train, array_val, array_test, num_features, chosen_transformer):
    
    num_features = num_features - 1
    
    data_train = array_train
    X_train  = data_train[:,0:num_features]
    Y_train  = data_train[:,num_features]
    
    data_val = array_val
    X_val  = data_val[:,0:num_features]
    Y_val  = data_val[:,num_features]
    
    data_test = array_test
    X_test  = data_test[:,0:num_features]
    Y_test  = data_test[:,num_features]
    
    transformer_train = chosen_transformer
    transformer_train = transformer_train.fit_transform(X_train, Y_train)
    
    transformer_val = chosen_transformer
    transformer_val = transformer_val.fit_transform(X_val, Y_val)
    
    transformer_test = chosen_transformer
    transformer_test = transformer_test.fit_transform(X_test, Y_test)
    return transformer_train, transformer_val, transformer_test, Y_train, Y_val, Y_test

def GenerateModelDataFromTransform(array_train, array_val, array_test, y_train, y_val, y_test, num_features, chosen_model):
    
    num_features = num_features - 1
    
    X_train  = array_train
    Y_train = y_train
    
    X_val = array_val
    Y_val = y_val
    
    X_test  = array_test
    Y_test = y_test
    
    model = chosen_model
    model = model.fit(X_train, Y_train)
    prediction_train = model.predict(X_train)
    prediction_val = model.predict(X_val)
    prediction_test = model.predict(X_test)
    return model, prediction_train, prediction_val, prediction_test, Y_train, Y_val, Y_test

In [8]:
#display methods
def ExportTree(model):
    dot_data = tree.export_graphviz(model, out_file=None, 
                         #feature_names = ['', '', ''],  
                         class_names = ['spam', 'mail'],
                         filled=True, rounded=True,  
                         special_characters=True)  
    graph = graphviz.Source(dot_data)
    graph.format = 'png'
    graph.render("Filter_data") 
    return graph

def GenerateScores(array):
    AccScores = np.zeros(len(array), dtype=float)
    F1Scores = np.zeros(len(array), dtype=float)

    for i in range(0, len(array)):
        AccScores[i] = accuracy_score(array[i][0], array[i][1])
        F1Scores[i] = f1_score(array[i][0], array[i][1], average='weighted')
    
    return AccScores, F1Scores

def GenerateAccScores(array):
    AccScores = np.zeros(len(array), dtype=float)

    for i in range(0, len(array)):
        AccScores[i] = accuracy_score(array[i][0], array[i][1])
    
    return AccScores

#Extraction de primitive
def TracePlot(array_acc_X, array_acc_Y, array_f1_X, array_f1_Y, titre, titre_x, titre_y):        
    
    plt.plot(array_acc_X, array_acc_Y, 'ro')
    plt.plot(array_f1_X, array_f1_Y, 'g*')
    plt.xlabel(titre_x)
    plt.ylabel(titre_y)
    plt.legend(['Accuracy Score','F1 Score'])
    plt.title(titre)
    plt.grid(True)
    plt.show()

In [9]:
#model creations methods
def CreateDecisionTreeModel(depth):
    return tree.DecisionTreeClassifier(criterion='entropy', max_depth=depth, min_samples_leaf=1)

def CreateKNNModel(k, weight):
    return KNeighborsClassifier(n_neighbors=k, weights=weight)

def CreateRandomForestModel(depth):
    return RandomForestClassifier(criterion='entropy', max_depth=depth, random_state=0)

In [10]:
#Decision tree methods
def DecisionTree_Holdout(array):
    array_train, array_val, array_test = SplitVectorData_Holdout(array, 0.6)

    array_model_none, array_prediction_train_none, array_prediction_val_none, array_prediction_test_none, array_train_none, array_val_none, array_test_none = GenerateModelDataFromVector(array_train, array_val, array_test, len(array[0]), CreateDecisionTreeModel(None))
    array_model_3, array_prediction_train_3, array_prediction_val_3, array_prediction_test_3, array_train_3, array_val_3, array_test_3 = GenerateModelDataFromVector(array_train, array_val, array_test, len(array[0]), CreateDecisionTreeModel(3))
    array_model_5, array_prediction_train_5, array_prediction_val_5, array_prediction_test_5, array_train_5, array_val_5, array_test_5 = GenerateModelDataFromVector(array_train, array_val, array_test, len(array[0]), CreateDecisionTreeModel(5))
    array_model_10, array_prediction_train_10, array_prediction_val_10, array_prediction_test_10, array_train_10, array_val_10, array_test_10 = GenerateModelDataFromVector(array_train, array_val, array_test, len(array[0]), CreateDecisionTreeModel(10))

    validations = [[array_val_none, array_prediction_val_none],
    [array_val_3, array_prediction_val_3],
    [array_val_5, array_prediction_val_5],
    [array_val_10, array_prediction_val_10]]

    return GenerateScores(validations)

def DecisionTree_KFold(array,k):

    array_kfold_train, array_kfold_test = SplitVectorData_KFold(array, k, 0.2)
    all_accScores = np.zeros((k, 4), dtype=float)
    all_f1Scores = np.zeros((k, 4), dtype=float)

    for i in range(1, k):

        array_train = concatenateWithoutTestPortion(array_kfold_train, i)
        array_val = array_kfold_train[i]
        array_test = array_kfold_test

        array_model_none, array_prediction_train_none, array_prediction_val_none, array_prediction_test_none, array_train_none, array_val_none, array_test_none = GenerateModelDataFromVector(array_train, array_val, array_test, len(array[0]), CreateDecisionTreeModel(None))
        array_model_3, array_prediction_train_3, array_prediction_val_3, array_prediction_test_3, array_train_3, array_val_3, array_test_3 = GenerateModelDataFromVector(array_train, array_val, array_test, len(array[0]), CreateDecisionTreeModel(3))
        array_model_5, array_prediction_train_5, array_prediction_val_5, array_prediction_test_tree_5, array_train_5, array_val_5, array_test_5 = GenerateModelDataFromVector(array_train, array_val, array_test, len(array[0]), CreateDecisionTreeModel(5))
        array_model_10, array_prediction_train_10, array_prediction_val_10, array_prediction_test_tree_10, array_train_10, array_val_10, array_test_10 = GenerateModelDataFromVector(array_train, array_val, array_test, len(array[0]), CreateDecisionTreeModel(10))

        validations = [[array_val_none, array_prediction_val_none],
        [array_val_3, array_prediction_val_3],
        [array_val_5, array_prediction_val_5],
        [array_val_10, array_prediction_val_10]]

        Array_AccScores, Array_F1Scores = GenerateScores(validations)

        all_accScores[i] = Array_AccScores
        all_f1Scores[i] = Array_F1Scores

    accScores_mean = scores_mean(all_accScores)
    f1Scores_mean = scores_mean(all_f1Scores)
    
    return accScores_mean, f1Scores_mean

In [11]:
#Bayes Methods
def Bayes_Holdout(array, array_prob ,array_transform=None):
    array_train, array_val, array_test = SplitVectorData_Holdout(array, 0.6)
    if array_transform is not None:
        array_train_tr, array_val_tr, array_test_tr = SplitVectorData_Holdout(array_transform, 0.6)

    transform_train_mdlp, transform_val_mdlp, transform_test_mdlp, Y_train_mdlp, Y_val_mdlp, Y_test_mdlp = TransformData(array_train, array_val, array_test, len(array_train[0]), MDLP())
    transform_train_minmax, transform_val_minmax, transform_test_minmax, Y_train_minmax, Y_val_minmax, Y_test_minmax = TransformData(array_train, array_val, array_test, len(array_train[0]), MinMaxScaler())

    array_model_5, array_prediction_train_5, array_prediction_val_5, array_prediction_test_5, array_train_5, array_val_5, array_test_5 = GenerateModelDataFromVector(array_train, array_val, array_test, len(array[0]), GaussianNB(priors=array_prob))
    if array_transform is not None:
        array_model_10, array_prediction_train_10, array_prediction_val_10, array_prediction_test_10, array_train_10, array_val_10, array_test_10 = GenerateModelDataFromVector(array_train_tr, array_val_tr, array_test_tr, len(array[0]), MultinomialNB(fit_prior=True, class_prior=array_prob))
    else:
        array_model_10, array_prediction_train_10, array_prediction_val_10, array_prediction_test_10, array_train_10, array_val_10, array_test_10 = GenerateModelDataFromVector(array_train, array_val, array_test, len(array[0]), MultinomialNB(fit_prior=True, class_prior=array_prob))

    array_model_mdlp, array_prediction_train_mdlp, array_prediction_val_mdlp, array_prediction_test_mdlp, array_train_mdlp, array_val_mdlp, array_test_mdlp = GenerateModelDataFromTransform(transform_train_mdlp, transform_val_mdlp, transform_test_mdlp, Y_train_mdlp, Y_val_mdlp, Y_test_mdlp, len(array[0]), MultinomialNB())
    array_model_minmax, array_prediction_train_minmax, array_prediction_val_minmax, array_prediction_test_minmax, array_train_minmax, array_val_minmax, array_test_minmax = GenerateModelDataFromTransform(transform_train_minmax, transform_val_minmax, transform_test_minmax, Y_train_minmax, Y_val_minmax, Y_test_minmax, len(array[0]), MultinomialNB())


    validations = [[array_val_5, array_prediction_val_5],
    [array_val_10, array_prediction_val_10],
    [array_val_mdlp, array_prediction_val_mdlp],
    [array_val_minmax, array_prediction_val_minmax]]

    return GenerateScores(validations)  
    
def Bayes_KFold(array, array_prob, k, array_transform=None):
    
    array_kfold_train, array_kfold_test = SplitVectorData_KFold(array, k, 0.2)
    if array_transform is not None:
        array_kfold_train_tr, array_kfold_test_tr = SplitVectorData_KFold(array_transform, k, 0.2)
    all_accScores = np.zeros((k, 4), dtype=float)
    all_f1Scores = np.zeros((k, 4), dtype=float)

    for i in range(1, k):

        array_train = concatenateWithoutTestPortion(array_kfold_train, i)
        array_val = array_kfold_train[i]
        array_test = array_kfold_test
        
        if array_transform is not None:
            array_train_tr = concatenateWithoutTestPortion(array_kfold_train_tr, i)
            array_val_tr = array_kfold_train_tr[i]
            array_test_tr = array_kfold_test_tr

        transform_train_mdlp, transform_val_mdlp, transform_test_mdlp, Y_train_mdlp, Y_val_mdlp, Y_test_mdlp = TransformData(array_train, array_val, array_test, len(array_train[0]), MDLP())
        transform_train_minmax, transform_val_minmax, transform_test_minmax, Y_train_minmax, Y_val_minmax, Y_test_minmax = TransformData(array_train, array_val, array_test, len(array_train[0]), MinMaxScaler())

        array_model_5, array_prediction_train_5, array_prediction_val_5, array_prediction_test_5, array_train_5, array_val_5, array_test_5 = GenerateModelDataFromVector(array_train, array_val, array_test, len(array[0]), GaussianNB(priors=array_prob))
        if array_transform is not None:
            array_model_10, array_prediction_train_10, array_prediction_val_10, array_prediction_test_10, array_train_10, array_val_10, array_test_10 = GenerateModelDataFromVector(array_train_tr, array_val_tr, array_test_tr, len(array[0]), MultinomialNB(fit_prior=True, class_prior=array_prob))
        else:
            array_model_10, array_prediction_train_10, array_prediction_val_10, array_prediction_test_10, array_train_10, array_val_10, array_test_10 = GenerateModelDataFromVector(array_train, array_val, array_test, len(array[0]), MultinomialNB(fit_prior=True, class_prior=array_prob))
        array_model_mdlp, array_prediction_train_mdlp, array_prediction_val_mdlp, array_prediction_test_mdlp, array_train_mdlp, array_val_mdlp, array_test_mdlp = GenerateModelDataFromTransform(transform_train_mdlp, transform_val_mdlp, transform_test_mdlp, Y_train_mdlp, Y_val_mdlp, Y_test_mdlp, len(array[0]), MultinomialNB())
        array_model_minmax, array_prediction_train_minmax, array_prediction_val_minmax, array_prediction_test_minmax, array_train_minmax, array_val_minmax, array_test_minmax = GenerateModelDataFromTransform(transform_train_minmax, transform_val_minmax, transform_test_minmax, Y_train_minmax, Y_val_minmax, Y_test_minmax, len(array[0]), MultinomialNB())


        validations = [[array_val_5, array_prediction_val_5],
                     [array_val_10, array_prediction_val_10],
                     [array_val_mdlp, array_prediction_val_mdlp],
                     [array_val_minmax, array_prediction_val_minmax]]

        Array_AccScores, Array_F1Scores = GenerateScores(validations)

        all_accScores[i] = Array_AccScores
        all_f1Scores[i] = Array_F1Scores 
        
    accScores_mean = scores_mean(all_accScores)
    f1Scores_mean = scores_mean(all_f1Scores)

    return accScores_mean, f1Scores_mean

In [12]:
#KNN Methods
def KNN_Holdout(array):
    array_train, array_val, array_test = SplitVectorData_Holdout(array, 0.6)

    array_model_3u, array_prediction_train_3u, array_prediction_val_3u, array_prediction_test_3u, array_train_3u, array_val_3u, array_test_3u = GenerateModelDataFromVector(array_train, array_val, array_test, len(array[0]), CreateKNNModel(3, 'uniform'))
    array_model_5u, array_prediction_train_5u, array_prediction_val_5u, array_prediction_test_5u, array_train_5u, array_val_5u, array_test_5u = GenerateModelDataFromVector(array_train, array_val, array_test, len(array[0]), CreateKNNModel(5, 'uniform'))
    array_model_10u, array_prediction_train_10u, array_prediction_val_10u, array_prediction_test_10u, array_train_10u, array_val_10u, array_test_10u = GenerateModelDataFromVector(array_train, array_val, array_test, len(array[0]), CreateKNNModel(10, 'uniform'))
    array_model_3d, array_prediction_train_3d, array_prediction_val_3d, array_prediction_test_3d, array_train_3d, array_val_3d, array_test_3d = GenerateModelDataFromVector(array_train, array_val, array_test, len(array[0]), CreateKNNModel(3, 'distance'))
    array_model_5d, array_prediction_train_5d, array_prediction_val_5d, array_prediction_test_5d, array_train_5d, array_val_5d, array_test_5d = GenerateModelDataFromVector(array_train, array_val, array_test, len(array[0]), CreateKNNModel(5, 'distance'))
    array_model_10d, array_prediction_train_10d, array_prediction_val_10d, array_prediction_test_10d, array_train_10d, array_val_10d, array_test_10d = GenerateModelDataFromVector(array_train, array_val, array_test, len(array[0]), CreateKNNModel(10, 'distance'))

    validations_uniform = [[array_val_3u, array_prediction_val_3u],
                        [array_val_5u, array_prediction_val_5u],
                        [array_val_10u, array_prediction_val_10u]]

    validations_distance = [[array_val_3d, array_prediction_val_3d],
                        [array_val_5d, array_prediction_val_5d],
                        [array_val_10d, array_prediction_val_10d]]

    accScores_uniform, f1Scores_uniform = GenerateScores(validations_uniform)
    accScores_distance, f1Scores_dsitance = GenerateScores(validations_distance)

    return accScores_uniform, f1Scores_uniform, accScores_distance, f1Scores_dsitance
    
def KNN_KFold(array, k):

    array_kfold_train, array_kfold_test = SplitVectorData_KFold(array, k, 0.2)
    all_accScores_uniform = np.zeros((k, 3), dtype=float)
    all_f1Scores_uniform = np.zeros((k, 3), dtype=float)
    all_accScores_distance = np.zeros((k, 3), dtype=float)
    all_f1Scores_distance = np.zeros((k, 3), dtype=float)

    for i in range(1, k):

        array_train = concatenateWithoutTestPortion(array_kfold_train, i)
        array_val = array_kfold_train[i]
        array_test = array_kfold_test

        array_model_3u, array_prediction_train_3u, array_prediction_val_3u, array_prediction_test_3u, array_train_3u, array_val_3u, array_test_3u = GenerateModelDataFromVector(array_train, array_val, array_test, len(array[0]), CreateKNNModel(3, 'uniform'))
        array_model_5u, array_prediction_train_5u, array_prediction_val_5u, array_prediction_test_5u, array_train_5u, array_val_5u, array_test_5u = GenerateModelDataFromVector(array_train, array_val, array_test, len(array[0]), CreateKNNModel(5, 'uniform'))
        array_model_10u, array_prediction_train_10u, array_prediction_val_10u, array_prediction_test_10u, array_train_10u, array_val_10u, array_test_10u = GenerateModelDataFromVector(array_train, array_val, array_test, len(array[0]), CreateKNNModel(10, 'uniform'))
        array_model_3d, array_prediction_train_3d, array_prediction_val_3d, array_prediction_test_3d, array_train_3d, array_val_3d, array_test_3d = GenerateModelDataFromVector(array_train, array_val, array_test, len(array[0]), CreateKNNModel(3, 'distance'))
        array_model_5d, array_prediction_train_5d, array_prediction_val_5d, array_prediction_test_5d, array_train_5d, array_val_5d, array_test_5d = GenerateModelDataFromVector(array_train, array_val, array_test, len(array[0]), CreateKNNModel(5, 'distance'))
        array_model_10d, array_prediction_train_10d, array_prediction_val_10d, array_prediction_test_10d, array_train_10d, array_val_10d, array_test_10d = GenerateModelDataFromVector(array_train, array_val, array_test, len(array[0]), CreateKNNModel(10, 'distance'))

        validations_uniform = [[array_val_3u, array_prediction_val_3u],
        [array_val_5u, array_prediction_val_5u],
        [array_val_10u, array_prediction_val_10u]]

        validations_distance = [[array_val_3d, array_prediction_val_3d],
        [array_val_5d, array_prediction_val_5d],
        [array_val_10d, array_prediction_val_10d]]

        Array_AccScores_uniform, Array_F1Scores_uniform = GenerateScores(validations_uniform)
        Array_AccScores_distance, Array_F1Scores_distance = GenerateScores(validations_distance)

        all_accScores_uniform[i] = Array_AccScores_uniform
        all_f1Scores_uniform[i] = Array_F1Scores_uniform
        all_accScores_distance[i] = Array_AccScores_distance
        all_f1Scores_distance[i] = Array_F1Scores_distance

    accScores_mean_uniform = scores_mean(all_accScores_uniform)
    f1Scores_mean_uniform = scores_mean(all_f1Scores_uniform)
    accScores_mean_distance = scores_mean(all_accScores_distance)
    f1Scores_mean_distance = scores_mean(all_f1Scores_distance)

    return accScores_mean_uniform, f1Scores_mean_uniform, accScores_mean_distance, f1Scores_mean_distance

In [13]:
#Random forest methods
def RandomForest_Holdout(array):
    array_train, array_val, array_test = SplitVectorData_Holdout(array, 0.6)

    array_model_none, array_prediction_train_none, array_prediction_val_none, array_prediction_test_none, array_train_none, array_val_none, array_test_none = GenerateModelDataFromVector(array_train, array_val, array_test, len(array[0]), CreateRandomForestModel(None))
    array_model_3, array_prediction_train_3, array_prediction_val_3, array_prediction_test_3, array_train_3, array_val_3, array_test_3 = GenerateModelDataFromVector(array_train, array_val, array_test, len(array[0]), CreateRandomForestModel(3))
    array_model_5, array_prediction_train_5, array_prediction_val_5, array_prediction_test_5, array_train_5, array_val_5, array_test_5 = GenerateModelDataFromVector(array_train, array_val, array_test, len(array[0]), CreateRandomForestModel(5))
    array_model_10, array_prediction_train_10, array_prediction_val_10, array_prediction_test_10, array_train_10, array_val_10, array_test_10 = GenerateModelDataFromVector(array_train, array_val, array_test, len(array[0]), CreateRandomForestModel(10))

    validations = [[array_val_none, array_prediction_val_none],
    [array_val_3, array_prediction_val_3],
    [array_val_5, array_prediction_val_5],
    [array_val_10, array_prediction_val_10]]

    return GenerateScores(validations)

def RandomForest_KFold(array,k):

    array_kfold_train, array_kfold_test = SplitVectorData_KFold(array, k, 0.2)
    all_accScores = np.zeros((k, 4), dtype=float)
    all_f1Scores = np.zeros((k, 4), dtype=float)

    for i in range(1, k):

        array_train = concatenateWithoutTestPortion(array_kfold_train, i)
        array_val = array_kfold_train[i]
        array_test = array_kfold_test

        array_model_none, array_prediction_train_none, array_prediction_val_none, array_prediction_test_none, array_train_none, array_val_none, array_test_none = GenerateModelDataFromVector(array_train, array_val, array_test, len(array[0]), CreateRandomForestModel(None))
        array_model_3, array_prediction_train_3, array_prediction_val_3, array_prediction_test_3, array_train_3, array_val_3, array_test_3 = GenerateModelDataFromVector(array_train, array_val, array_test, len(array[0]), CreateRandomForestModel(3))
        array_model_5, array_prediction_train_5, array_prediction_val_5, array_prediction_test_tree_5, array_train_5, array_val_5, array_test_5 = GenerateModelDataFromVector(array_train, array_val, array_test, len(array[0]), CreateRandomForestModel(5))
        array_model_10, array_prediction_train_10, array_prediction_val_10, array_prediction_test_tree_10, array_train_10, array_val_10, array_test_10 = GenerateModelDataFromVector(array_train, array_val, array_test, len(array[0]), CreateRandomForestModel(10))

        validations = [[array_val_none, array_prediction_val_none],
        [array_val_3, array_prediction_val_3],
        [array_val_5, array_prediction_val_5],
        [array_val_10, array_prediction_val_10]]

        Array_AccScores, Array_F1Scores = GenerateScores(validations)

        all_accScores[i] = Array_AccScores
        all_f1Scores[i] = Array_F1Scores

    accScores_mean = scores_mean(all_accScores)
    f1Scores_mean = scores_mean(all_f1Scores)
    
    return accScores_mean, f1Scores_mean

In [14]:
# accScores_filter_tree_holdout, f1Scores_filter_tree_holdout = DecisionTree_Holdout(Filter)
# accScores_filter_tree_kfold, f1Scores_filter_tree_kfold = DecisionTree_KFold(Filter,10)
# accScores_filter_bayes_holdout, f1Scores_filter_bayes_holdout = Bayes_Holdout(Filter, [0.4003,0.5997])
# accScores_filter_bayes_kfold, f1Scores_filter_bayes_kfold = Bayes_KFold(Filter, [0.4003,0.5997], 10)
# accScores_filter_knn_holdout_uniform, f1Scores_filter_knn_holdout_uniform, accScores_filter_knn_holdout_distance, f1Scores_filter_knn_holdout_distance = KNN_Holdout(Filter)
# accScores_filter_knn_kfold_uniform, f1Scores_filter_knn_kfold_uniform, accScores_filter_knn_kfold_distance, f1Scores_filter_knn_kfold_distance = KNN_KFold(Filter, 10) 
# accScores_filter_test, f1Scores_filter_test = UseModelOnTestData(Filter, MultinomialNB(), MinMaxScaler())
# accScores_filter_forest_holdout, f1Scores_filter_forest_holdout = RandomForest_Holdout(Filter)
# accScores_filter_forest_kfold, f1Scores_filter_forest_kfold = RandomForest_KFold(Filter,10)

### Galaxies

In [15]:
# Get some galaxies
fid = open('galaxy_feature_vectors.csv', 'r') 

NbGalaxy = 16000 #16000
Galaxies = np.zeros((NbGalaxy, 7), dtype=np.float64)

count = 0
count_smooth = 0
count_spiral = 0

for line in fid:
    element = line.rstrip('\n').split(',')
    
    label = float(element[75])
    
    if label == 0.0 and count_smooth < NbGalaxy/2:
        count_smooth += 1     
    elif label == 1.0 and count_spiral < NbGalaxy/2:
        count_spiral += 1
    else:
        continue

    Galaxies[count] = [element[4], element[5], element[6], element[17], element[23], element[24], element[75]]
        
    count += 1
    if count >= NbGalaxy:
        break

fid.close() 

# ----- For debug -----
#print(count_smooth)
#print(count_spiral)
#print(count)
#print(Galaxies)
#----------------------

In [16]:
def NormalizeData(data):
    transform = MinMaxScaler()
    return transform.fit_transform(data, data[:,len(Galaxies[0])-1])

def GetGalaxiesClassProbabilities():
    count_smooth = 0
    count_spiral = 0
    
    fid = open('galaxy_feature_vectors.csv', 'r') 
    for line in fid:
        element = line.rstrip('\n').split(',')

        label = float(element[75])

        if label == 0.0:
            count_smooth += 1     
        elif label == 1.0:
            count_spiral += 1
    return [count_smooth/(count_smooth+count_spiral), count_spiral/(count_smooth+count_spiral)]

In [34]:
def SplitLabelsFromPrimitives(data_train, data_val, data_test):
    num_features = len(data_train[0]) - 1
    
    X_train  = data_train[:,0:num_features]
    Y_train  = data_train[:,num_features]

    X_val  = data_val[:,0:num_features]
    Y_val  = data_val[:,num_features]

    X_test  = data_test[:,0:num_features]
    Y_test  = data_test[:,num_features]
    return X_train, Y_train, X_val, Y_val, X_test, Y_test

def GenerateModel(data_train, data_val, data_test, layers, perceptrons, epochs, learnRate, name):
    
    #Split labels from the primitives
    X_train, Y_train, X_val, Y_val, X_test, Y_test = SplitLabelsFromPrimitives(data_train, data_val, data_test)
    
    #Create model
    model = Sequential()

    #Add layers
    model.add(Dense(units=perceptrons, activation='sigmoid', input_shape=(len(X_train[0]),)))
    model.add(Dropout(0.2))
    for i in range(0, layers - 2):
        model.add(Dense(units=perceptrons, activation='sigmoid'))
        model.add(Dropout(0.2))
    model.add(Dense(units=2, activation='sigmoid'))
    
    #Set optimizers and compile
    sgd = opt.SGD(lr=learnRate, decay=0, momentum=0, nesterov=False)
    model.compile(loss='sparse_categorical_crossentropy', 
              optimizer= sgd,
              metrics=['accuracy'])
    
    #Use TensorBoard
    tb_callback = tf.keras.callbacks.TensorBoard(log_dir='./logs/'+name)
    
    #Train
    model.fit(X_train, Y_train, validation_data=(X_val, Y_val), epochs=epochs, batch_size=100, callbacks=[tb_callback])
    
    model.evaluate(X_test, Y_test)
    
    Y_probs = model.predict(X_test)
    
    Y_pred = Y_probs.argmax(axis=1)
    
    #plt.scatter(X_test[:, 0], X_test[:, 1], c=Y_pred)
    #plt.scatter(X_test[:, 0], X_test[:, 1], c=Y_test)

    return Y_test, Y_pred

def Neurone_Holdout(array):
    array_train, array_val, array_test = SplitVectorData_Holdout(array, 0.6)
    
    AccScores = np.zeros(13, dtype=float)
    
    #base
    y_true_base, y_pred_base = GenerateModel(array_train, array_val, array_test, 4, 100, 60, 0.0005, 'base')
    
    #perceptrons
    y_true_perc1, y_pred_perc1 = GenerateModel(array_train, array_val, array_test, 4, 50, 60, 0.0005, 'perc1')
    y_true_perc2, y_pred_perc2 = GenerateModel(array_train, array_val, array_test, 4, 300, 60, 0.0005, 'perc2')
    y_true_perc3, y_pred_perc3 = GenerateModel(array_train, array_val, array_test, 4, 600, 60, 0.0005, 'perc3')

    #epochs
    y_true_epoch1, y_pred_epoch1 = GenerateModel(array_train, array_val, array_test, 4, 100, 30, 0.0005, 'epoch1')
    y_true_epoch2, y_pred_epoch2 = GenerateModel(array_train, array_val, array_test, 4, 100, 120, 0.0005, 'epoch2')
    y_true_epoch3, y_pred_epoch3 = GenerateModel(array_train, array_val, array_test, 4, 100, 240, 0.0005, 'epoch3')
    
    #layers
    y_true_layer1, y_pred_layer1 = GenerateModel(array_train, array_val, array_test, 6, 100, 60, 0.0005, 'layer1')
    y_true_layer2, y_pred_layer2 = GenerateModel(array_train, array_val, array_test, 12, 100, 60, 0.0005, 'layer2')
    y_true_layer3, y_pred_layer3 = GenerateModel(array_train, array_val, array_test, 24, 100, 60, 0.0005, 'layer3')
    
    #learning rate
    y_true_lr1, y_pred_lr1 = GenerateModel(array_train, array_val, array_test, 4, 100, 60, 0.1, 'lr1')
    y_true_lr2, y_pred_lr2 = GenerateModel(array_train, array_val, array_test, 4, 100, 60, 0.5, 'lr2')
    y_true_lr3, y_pred_lr3 = GenerateModel(array_train, array_val, array_test, 4, 100, 60, 1, 'lr3')

    validations = [[y_true_base, y_pred_base],
    [y_true_perc1, y_pred_perc1],
    [y_true_perc2, y_pred_perc2],
    [y_true_perc3, y_pred_perc3],
    [y_true_epoch1, y_pred_epoch1],
    [y_true_epoch2, y_pred_epoch2],
    [y_true_epoch3, y_pred_epoch3],
    [y_true_layer1, y_pred_epoch1],
    [y_true_layer2, y_pred_layer2],
    [y_true_layer3, y_pred_layer3],
    [y_true_lr1, y_pred_lr1],
    [y_true_lr2, y_pred_lr2],
    [y_true_lr3, y_pred_lr3]]
    
    return GenerateAccScores(validations)
    
def Neurone_KFold(array,k):

    array_kfold_train, array_kfold_test = SplitVectorData_KFold(array, k, 0.2)
    all_accScores = np.zeros((k, 13), dtype=float)
    all_f1Scores = np.zeros((k, 13), dtype=float)

    for i in range(1, k):

        array_train = concatenateWithoutTestPortion(array_kfold_train, i)
        array_val = array_kfold_train[i]
        array_test = array_kfold_test
        
        #base
        y_true_base, y_pred_base = GenerateModel(array_train, array_val, array_test, 4, 100, 60, 0.0005, 'kfold/k'+str(i)+'_base')

        #perceptrons
        y_true_perc1, y_pred_perc1 = GenerateModel(array_train, array_val, array_test, 4, 50, 60, 0.0005, 'kfold/k'+str(i)+'_perc1')
        y_true_perc2, y_pred_perc2 = GenerateModel(array_train, array_val, array_test, 4, 300, 60, 0.0005, 'kfold/k'+str(i)+'_perc2')
        y_true_perc3, y_pred_perc3 = GenerateModel(array_train, array_val, array_test, 4, 600, 60, 0.0005, 'kfold/k'+str(i)+'_perc3')

        #epochs
        y_true_epoch1, y_pred_epoch1 = GenerateModel(array_train, array_val, array_test, 4, 100, 30, 0.0005, 'kfold/k'+str(i)+'_epoch1')
        y_true_epoch2, y_pred_epoch2 = GenerateModel(array_train, array_val, array_test, 4, 100, 120, 0.0005, 'kfold/k'+str(i)+'_epoch2')
        y_true_epoch3, y_pred_epoch3 = GenerateModel(array_train, array_val, array_test, 4, 100, 240, 0.0005, 'kfold/k'+str(i)+'_epoch3')

        #layers
        y_true_layer1, y_pred_layer1 = GenerateModel(array_train, array_val, array_test, 6, 100, 60, 0.0005, 'kfold/k'+str(i)+'_layer1')
        y_true_layer2, y_pred_layer2 = GenerateModel(array_train, array_val, array_test, 12, 100, 60, 0.0005, 'kfold/k'+str(i)+'_layer2')
        y_true_layer3, y_pred_layer3 = GenerateModel(array_train, array_val, array_test, 24, 100, 60, 0.0005, 'kfold/k'+str(i)+'_layer3')

        #learning rate
        y_true_lr1, y_pred_lr1 = GenerateModel(array_train, array_val, array_test, 4, 100, 60, 0.1, 'kfold/k'+str(i)+'_lr1')
        y_true_lr2, y_pred_lr2 = GenerateModel(array_train, array_val, array_test, 4, 100, 60, 0.5, 'kfold/k'+str(i)+'_lr2')
        y_true_lr3, y_pred_lr3 = GenerateModel(array_train, array_val, array_test, 4, 100, 60, 1, 'kfold/k'+str(i)+'_lr3')

        validations = [[y_true_base, y_pred_base],
        [y_true_perc1, y_pred_perc1],
        [y_true_perc2, y_pred_perc2],
        [y_true_perc3, y_pred_perc3],
        [y_true_epoch1, y_pred_epoch1],
        [y_true_epoch2, y_pred_epoch2],
        [y_true_epoch3, y_pred_epoch3],
        [y_true_layer1, y_pred_epoch1],
        [y_true_layer2, y_pred_layer2],
        [y_true_layer3, y_pred_layer3],
        [y_true_lr1, y_pred_lr1],
        [y_true_lr2, y_pred_lr2],
        [y_true_lr3, y_pred_lr3]]

        Array_AccScores = GenerateAccScores(validations)

        all_accScores[i] = Array_AccScores

    accScores_mean = scores_mean(all_accScores)
    
    return accScores_mean


In [18]:
accScores_neurone_holdout = Neurone_Holdout(Galaxies)

Train on 9600 samples, validate on 3200 samples
Epoch 1/60
9600/9600 [==============================] - 0s 44us/step - loss: 0.7902 - acc: 0.5197 - val_loss: 0.7596 - val_acc: 0.5181
Epoch 2/60
9600/9600 [==============================] - 0s 25us/step - loss: 0.7576 - acc: 0.5176 - val_loss: 0.7335 - val_acc: 0.5181
Epoch 3/60
9600/9600 [==============================] - 0s 29us/step - loss: 0.7366 - acc: 0.5201 - val_loss: 0.7177 - val_acc: 0.5181
Epoch 4/60
9600/9600 [==============================] - 0s 28us/step - loss: 0.7258 - acc: 0.5191 - val_loss: 0.7080 - val_acc: 0.5181
Epoch 5/60
9600/9600 [==============================] - 0s 27us/step - loss: 0.7155 - acc: 0.5182 - val_loss: 0.7021 - val_acc: 0.5181
Epoch 6/60
9600/9600 [==============================] - 0s 28us/step - loss: 0.7112 - acc: 0.5135 - val_loss: 0.6985 - val_acc: 0.5181
Epoch 7/60
9600/9600 [==============================] - 0s 26us/step - loss: 0.7079 - acc: 0.5115 - val_loss: 0.6963 - val_acc: 0.5181
Epoch 8

3200/3200 [==============================] - 0s 28us/step
Train on 9600 samples, validate on 3200 samples
Epoch 1/60
9600/9600 [==============================] - 0s 35us/step - loss: 0.7049 - acc: 0.5146 - val_loss: 0.6952 - val_acc: 0.5181
Epoch 2/60
9600/9600 [==============================] - 0s 20us/step - loss: 0.7042 - acc: 0.5160 - val_loss: 0.6944 - val_acc: 0.5181
Epoch 3/60
9600/9600 [==============================] - 0s 21us/step - loss: 0.7031 - acc: 0.5111 - val_loss: 0.6938 - val_acc: 0.5181
Epoch 4/60
9600/9600 [==============================] - 0s 18us/step - loss: 0.7039 - acc: 0.5080 - val_loss: 0.6934 - val_acc: 0.5181
Epoch 5/60
9600/9600 [==============================] - 0s 20us/step - loss: 0.7020 - acc: 0.5101 - val_loss: 0.6930 - val_acc: 0.5181
Epoch 6/60
9600/9600 [==============================] - 0s 19us/step - loss: 0.7009 - acc: 0.5098 - val_loss: 0.6928 - val_acc: 0.5181
Epoch 7/60
9600/9600 [==============================] - 0s 20us/step - loss: 0.7028 

3200/3200 [==============================] - 0s 25us/step
Train on 9600 samples, validate on 3200 samples
Epoch 1/60
9600/9600 [==============================] - 1s 74us/step - loss: 0.7140 - acc: 0.4967 - val_loss: 0.6972 - val_acc: 0.4819
Epoch 2/60
9600/9600 [==============================] - 1s 61us/step - loss: 0.7066 - acc: 0.4999 - val_loss: 0.6941 - val_acc: 0.3703
Epoch 3/60
9600/9600 [==============================] - 1s 57us/step - loss: 0.7080 - acc: 0.4939 - val_loss: 0.6935 - val_acc: 0.5181
Epoch 4/60
9600/9600 [==============================] - 0s 50us/step - loss: 0.7038 - acc: 0.5075 - val_loss: 0.6934 - val_acc: 0.5181
Epoch 5/60
9600/9600 [==============================] - 0s 51us/step - loss: 0.7089 - acc: 0.4900 - val_loss: 0.6934 - val_acc: 0.5181
Epoch 6/60
9600/9600 [==============================] - 1s 60us/step - loss: 0.7054 - acc: 0.4980 - val_loss: 0.6934 - val_acc: 0.5181
Epoch 7/60
9600/9600 [==============================] - 1s 58us/step - loss: 0.7044 

9600/9600 [==============================] - 0s 50us/step - loss: 0.6977 - acc: 0.4998 - val_loss: 0.6928 - val_acc: 0.5181
Epoch 60/60
3200/3200 [==============================] - 0s 32us/step
Train on 9600 samples, validate on 3200 samples
Epoch 1/60
9600/9600 [==============================] - 2s 196us/step - loss: 0.7281 - acc: 0.4887 - val_loss: 0.6955 - val_acc: 0.4819
Epoch 2/60
9600/9600 [==============================] - 2s 166us/step - loss: 0.7074 - acc: 0.4952 - val_loss: 0.6930 - val_acc: 0.5181
Epoch 3/60
9600/9600 [==============================] - 2s 166us/step - loss: 0.7052 - acc: 0.5007 - val_loss: 0.6929 - val_acc: 0.5181
Epoch 4/60
9600/9600 [==============================] - 2s 167us/step - loss: 0.7048 - acc: 0.5023 - val_loss: 0.6929 - val_acc: 0.5181
Epoch 5/60
9600/9600 [==============================] - 2s 181us/step - loss: 0.7055 - acc: 0.5016 - val_loss: 0.6929 - val_acc: 0.5181
Epoch 6/60
9600/9600 [==============================] - 2s 177us/step - loss: 

9600/9600 [==============================] - 2s 167us/step - loss: 0.6944 - acc: 0.5055 - val_loss: 0.6925 - val_acc: 0.5181
Epoch 59/60
9600/9600 [==============================] - 2s 169us/step - loss: 0.6941 - acc: 0.5155 - val_loss: 0.6925 - val_acc: 0.5181
Epoch 60/60
3200/3200 [==============================] - 0s 79us/step
Train on 9600 samples, validate on 3200 samples
Epoch 1/30
9600/9600 [==============================] - 1s 54us/step - loss: 0.6996 - acc: 0.5194 - val_loss: 0.6954 - val_acc: 0.5181
Epoch 2/30
9600/9600 [==============================] - 0s 30us/step - loss: 0.7006 - acc: 0.5177 - val_loss: 0.6947 - val_acc: 0.5181
Epoch 3/30
9600/9600 [==============================] - 0s 31us/step - loss: 0.7007 - acc: 0.5150 - val_loss: 0.6942 - val_acc: 0.5181
Epoch 4/30
9600/9600 [==============================] - 0s 40us/step - loss: 0.7022 - acc: 0.5084 - val_loss: 0.6938 - val_acc: 0.5181
Epoch 5/30
9600/9600 [==============================] - 0s 46us/step - loss: 0.6

...

9600/9600 [==============================] - 1s 75us/step - loss: 0.7016 - acc: 0.5025 - val_loss: 0.6925 - val_acc: 0.5181
Epoch 22/60
9600/9600 [==============================] - 1s 78us/step - loss: 0.6986 - acc: 0.5125 - val_loss: 0.6925 - val_acc: 0.5181
Epoch 23/60
9600/9600 [==============================] - 1s 78us/step - loss: 0.7003 - acc: 0.5044 - val_loss: 0.6925 - val_acc: 0.5181
Epoch 24/60
9600/9600 [==============================] - 1s 78us/step - loss: 0.7002 - acc: 0.5112 - val_loss: 0.6925 - val_acc: 0.5181
Epoch 25/60
9600/9600 [==============================] - 1s 80us/step - loss: 0.7008 - acc: 0.5091 - val_loss: 0.6925 - val_acc: 0.5181
Epoch 26/60
9600/9600 [==============================] - 1s 78us/step - loss: 0.7005 - acc: 0.5070 - val_loss: 0.6925 - val_acc: 0.5181
Epoch 27/60
9600/9600 [==============================] - 1s 83us/step - loss: 0.7009 - acc: 0.5017 - val_loss: 0.6925 - val_acc: 0.5181
Epoch 28/60
9600/9600 [==============================] - 1s 

9600/9600 [==============================] - 2s 157us/step - loss: 0.6966 - acc: 0.5075 - val_loss: 0.6925 - val_acc: 0.5181
Epoch 21/60
9600/9600 [==============================] - 2s 159us/step - loss: 0.6974 - acc: 0.5029 - val_loss: 0.6925 - val_acc: 0.5181
Epoch 22/60
9600/9600 [==============================] - 2s 158us/step - loss: 0.6966 - acc: 0.5106 - val_loss: 0.6925 - val_acc: 0.5181
Epoch 23/60
9600/9600 [==============================] - 1s 156us/step - loss: 0.6961 - acc: 0.5129 - val_loss: 0.6925 - val_acc: 0.5181
Epoch 24/60
9600/9600 [==============================] - 2s 164us/step - loss: 0.6958 - acc: 0.5162 - val_loss: 0.6925 - val_acc: 0.5181
Epoch 25/60
9600/9600 [==============================] - 2s 159us/step - loss: 0.6958 - acc: 0.5049 - val_loss: 0.6925 - val_acc: 0.5181
Epoch 26/60
9600/9600 [==============================] - 2s 161us/step - loss: 0.6964 - acc: 0.5047 - val_loss: 0.6925 - val_acc: 0.5181
Epoch 27/60
9600/9600 [==============================

9600/9600 [==============================] - 0s 33us/step - loss: 0.6921 - acc: 0.5239 - val_loss: 0.6915 - val_acc: 0.5181
Epoch 20/60
9600/9600 [==============================] - 0s 28us/step - loss: 0.6918 - acc: 0.5212 - val_loss: 0.6915 - val_acc: 0.5181
Epoch 21/60
9600/9600 [==============================] - 0s 33us/step - loss: 0.6913 - acc: 0.5234 - val_loss: 0.6916 - val_acc: 0.5181
Epoch 22/60
9600/9600 [==============================] - 0s 23us/step - loss: 0.6920 - acc: 0.5226 - val_loss: 0.6913 - val_acc: 0.5181
Epoch 23/60
9600/9600 [==============================] - 0s 26us/step - loss: 0.6914 - acc: 0.5201 - val_loss: 0.6911 - val_acc: 0.5181
Epoch 24/60
9600/9600 [==============================] - 0s 27us/step - loss: 0.6911 - acc: 0.5270 - val_loss: 0.6910 - val_acc: 0.5181
Epoch 25/60
9600/9600 [==============================] - 0s 31us/step - loss: 0.6913 - acc: 0.5199 - val_loss: 0.6908 - val_acc: 0.5181
Epoch 26/60
9600/9600 [==============================] - 0s 

9600/9600 [==============================] - 0s 33us/step - loss: 0.5697 - acc: 0.7253 - val_loss: 0.6463 - val_acc: 0.6572
Epoch 19/60
9600/9600 [==============================] - 0s 31us/step - loss: 0.5607 - acc: 0.7292 - val_loss: 0.5247 - val_acc: 0.7703
Epoch 20/60
9600/9600 [==============================] - 0s 29us/step - loss: 0.5514 - acc: 0.7355 - val_loss: 0.5738 - val_acc: 0.7009
Epoch 21/60
9600/9600 [==============================] - 0s 28us/step - loss: 0.5460 - acc: 0.7498 - val_loss: 0.5919 - val_acc: 0.6928
Epoch 22/60
9600/9600 [==============================] - 0s 35us/step - loss: 0.5318 - acc: 0.7529 - val_loss: 0.5049 - val_acc: 0.7800
Epoch 23/60
9600/9600 [==============================] - 0s 27us/step - loss: 0.5294 - acc: 0.7582 - val_loss: 0.5018 - val_acc: 0.7837
Epoch 24/60
9600/9600 [==============================] - 0s 31us/step - loss: 0.5308 - acc: 0.7553 - val_loss: 0.5301 - val_acc: 0.7400
Epoch 25/60
9600/9600 [==============================] - 0s 

9600/9600 [==============================] - 0s 25us/step - loss: 0.5497 - acc: 0.7416 - val_loss: 0.7053 - val_acc: 0.6028
Epoch 18/60
9600/9600 [==============================] - 0s 29us/step - loss: 0.5560 - acc: 0.7318 - val_loss: 0.5171 - val_acc: 0.7609
Epoch 19/60
9600/9600 [==============================] - 0s 27us/step - loss: 0.5418 - acc: 0.7444 - val_loss: 0.5108 - val_acc: 0.7759
Epoch 20/60
9600/9600 [==============================] - 0s 27us/step - loss: 0.5405 - acc: 0.7449 - val_loss: 0.5560 - val_acc: 0.7206
Epoch 21/60
9600/9600 [==============================] - 0s 28us/step - loss: 0.5288 - acc: 0.7519 - val_loss: 0.5206 - val_acc: 0.7684
Epoch 22/60
9600/9600 [==============================] - 0s 28us/step - loss: 0.5284 - acc: 0.7574 - val_loss: 0.4926 - val_acc: 0.7825
Epoch 23/60
9600/9600 [==============================] - 0s 29us/step - loss: 0.5181 - acc: 0.7617 - val_loss: 0.4880 - val_acc: 0.7844
Epoch 24/60
9600/9600 [==============================] - 0s 

In [35]:
accScores_neurone_kfold = Neurone_KFold(Galaxies, 3)

Train on 11520 samples, validate on 1280 samples
Epoch 1/60
11520/11520 [==============================] - 1s 67us/step - loss: 0.7885 - acc: 0.4267 - val_loss: 0.7395 - val_acc: 0.4805
Epoch 2/60
11520/11520 [==============================] - 0s 26us/step - loss: 0.7635 - acc: 0.4269 - val_loss: 0.7242 - val_acc: 0.4805
Epoch 3/60
11520/11520 [==============================] - 0s 27us/step - loss: 0.7441 - acc: 0.4302 - val_loss: 0.7142 - val_acc: 0.4805
Epoch 4/60
11520/11520 [==============================] - 0s 28us/step - loss: 0.7296 - acc: 0.4319 - val_loss: 0.7076 - val_acc: 0.4805
Epoch 5/60
11520/11520 [==============================] - 0s 29us/step - loss: 0.7220 - acc: 0.4373 - val_loss: 0.7029 - val_acc: 0.4805
Epoch 6/60
11520/11520 [==============================] - 0s 30us/step - loss: 0.7156 - acc: 0.4424 - val_loss: 0.6996 - val_acc: 0.4805
Epoch 7/60
11520/11520 [==============================] - 0s 29us/step - loss: 0.7087 - acc: 0.4517 - val_loss: 0.6973 - val_acc:

3200/3200 [==============================] - 0s 37us/step
Train on 11520 samples, validate on 1280 samples
Epoch 1/60
11520/11520 [==============================] - 1s 58us/step - loss: 0.7269 - acc: 0.4472 - val_loss: 0.7018 - val_acc: 0.4805
Epoch 2/60
11520/11520 [==============================] - 0s 17us/step - loss: 0.7177 - acc: 0.4632 - val_loss: 0.6978 - val_acc: 0.4805
Epoch 3/60
11520/11520 [==============================] - 0s 18us/step - loss: 0.7094 - acc: 0.4781 - val_loss: 0.6953 - val_acc: 0.4805
Epoch 4/60
11520/11520 [==============================] - 0s 18us/step - loss: 0.7058 - acc: 0.4875 - val_loss: 0.6937 - val_acc: 0.4805
Epoch 5/60
11520/11520 [==============================] - 0s 18us/step - loss: 0.7024 - acc: 0.4977 - val_loss: 0.6928 - val_acc: 0.5195
Epoch 6/60
11520/11520 [==============================] - 0s 19us/step - loss: 0.6982 - acc: 0.5051 - val_loss: 0.6924 - val_acc: 0.5195
Epoch 7/60
11520/11520 [==============================] - 0s 18us/step 

11520/11520 [==============================] - 0s 21us/step - loss: 0.6876 - acc: 0.5593 - val_loss: 0.6981 - val_acc: 0.5195
Epoch 59/60
11520/11520 [==============================] - 0s 26us/step - loss: 0.6877 - acc: 0.5604 - val_loss: 0.6980 - val_acc: 0.5195
Epoch 60/60
3200/3200 [==============================] - 0s 26us/step
Train on 11520 samples, validate on 1280 samples
Epoch 1/60
11520/11520 [==============================] - 1s 102us/step - loss: 0.6926 - acc: 0.5220 - val_loss: 0.6925 - val_acc: 0.5195
Epoch 2/60
11520/11520 [==============================] - 1s 56us/step - loss: 0.6899 - acc: 0.5369 - val_loss: 0.6930 - val_acc: 0.5195
Epoch 3/60
11520/11520 [==============================] - 1s 58us/step - loss: 0.6881 - acc: 0.5501 - val_loss: 0.6937 - val_acc: 0.5195
Epoch 4/60
11520/11520 [==============================] - 1s 57us/step - loss: 0.6884 - acc: 0.5503 - val_loss: 0.6944 - val_acc: 0.5195
Epoch 5/60
11520/11520 [==============================] - 1s 59us/st

Epoch 57/60
11520/11520 [==============================] - 1s 51us/step - loss: 0.6848 - acc: 0.5723 - val_loss: 0.6968 - val_acc: 0.5195
Epoch 58/60
11520/11520 [==============================] - 1s 50us/step - loss: 0.6845 - acc: 0.5706 - val_loss: 0.6968 - val_acc: 0.5195
Epoch 59/60
11520/11520 [==============================] - 1s 49us/step - loss: 0.6851 - acc: 0.5707 - val_loss: 0.6968 - val_acc: 0.5195
Epoch 60/60
3200/3200 [==============================] - 0s 52us/step
Train on 11520 samples, validate on 1280 samples
Epoch 1/60
11520/11520 [==============================] - 2s 204us/step - loss: 0.7118 - acc: 0.4832 - val_loss: 0.6922 - val_acc: 0.5195
Epoch 2/60
11520/11520 [==============================] - 2s 153us/step - loss: 0.6930 - acc: 0.5384 - val_loss: 0.6951 - val_acc: 0.5195
Epoch 3/60
11520/11520 [==============================] - 2s 174us/step - loss: 0.6886 - acc: 0.5524 - val_loss: 0.6971 - val_acc: 0.5195
Epoch 4/60
11520/11520 [=============================

11520/11520 [==============================] - 2s 141us/step - loss: 0.6843 - acc: 0.5734 - val_loss: 0.6965 - val_acc: 0.5195
Epoch 56/60
11520/11520 [==============================] - 2s 140us/step - loss: 0.6834 - acc: 0.5741 - val_loss: 0.6967 - val_acc: 0.5195
Epoch 57/60
11520/11520 [==============================] - 2s 138us/step - loss: 0.6849 - acc: 0.5718 - val_loss: 0.6966 - val_acc: 0.5195
Epoch 58/60
11520/11520 [==============================] - 2s 142us/step - loss: 0.6845 - acc: 0.5727 - val_loss: 0.6965 - val_acc: 0.5195
Epoch 59/60
11520/11520 [==============================] - 2s 142us/step - loss: 0.6847 - acc: 0.5712 - val_loss: 0.6965 - val_acc: 0.5195
Epoch 60/60
3200/3200 [==============================] - 0s 85us/step
Train on 11520 samples, validate on 1280 samples
Epoch 1/30
11520/11520 [==============================] - 1s 72us/step - loss: 0.7428 - acc: 0.5736 - val_loss: 0.7735 - val_acc: 0.5195
Epoch 2/30
11520/11520 [==============================] - 0s 

...

11520/11520 [==============================] - 2s 173us/step - loss: 0.6946 - acc: 0.5478 - val_loss: 0.7026 - val_acc: 0.5062
Epoch 21/60
11520/11520 [==============================] - 2s 173us/step - loss: 0.6930 - acc: 0.5498 - val_loss: 0.7027 - val_acc: 0.5062
Epoch 22/60
11520/11520 [==============================] - 2s 164us/step - loss: 0.6914 - acc: 0.5561 - val_loss: 0.7027 - val_acc: 0.5062
Epoch 23/60
11520/11520 [==============================] - 2s 172us/step - loss: 0.6954 - acc: 0.5506 - val_loss: 0.7025 - val_acc: 0.5062
Epoch 24/60
11520/11520 [==============================] - 2s 171us/step - loss: 0.6951 - acc: 0.5492 - val_loss: 0.7023 - val_acc: 0.5062
Epoch 25/60
11520/11520 [==============================] - 2s 172us/step - loss: 0.6922 - acc: 0.5529 - val_loss: 0.7025 - val_acc: 0.5062
Epoch 26/60
11520/11520 [==============================] - 2s 172us/step - loss: 0.6930 - acc: 0.5543 - val_loss: 0.7025 - val_acc: 0.5062
Epoch 27/60
11520/11520 [==============

11520/11520 [==============================] - 4s 371us/step - loss: 0.6959 - acc: 0.5457 - val_loss: 0.7031 - val_acc: 0.5062
Epoch 19/60
11520/11520 [==============================] - 4s 367us/step - loss: 0.6933 - acc: 0.5522 - val_loss: 0.7032 - val_acc: 0.5062
Epoch 20/60
11520/11520 [==============================] - 4s 379us/step - loss: 0.6949 - acc: 0.5485 - val_loss: 0.7033 - val_acc: 0.5062
Epoch 21/60
11520/11520 [==============================] - 4s 378us/step - loss: 0.6939 - acc: 0.5538 - val_loss: 0.7034 - val_acc: 0.5062
Epoch 22/60
11520/11520 [==============================] - 4s 377us/step - loss: 0.6954 - acc: 0.5451 - val_loss: 0.7033 - val_acc: 0.5062
Epoch 23/60
11520/11520 [==============================] - 4s 364us/step - loss: 0.6942 - acc: 0.5514 - val_loss: 0.7032 - val_acc: 0.5062
Epoch 24/60
11520/11520 [==============================] - 4s 374us/step - loss: 0.6935 - acc: 0.5530 - val_loss: 0.7031 - val_acc: 0.5062
Epoch 25/60
11520/11520 [==============

11520/11520 [==============================] - 1s 74us/step - loss: 0.6800 - acc: 0.5751 - val_loss: 0.7033 - val_acc: 0.5062
Epoch 17/60
11520/11520 [==============================] - 1s 71us/step - loss: 0.6797 - acc: 0.5751 - val_loss: 0.6973 - val_acc: 0.5062
Epoch 18/60
11520/11520 [==============================] - 1s 70us/step - loss: 0.6783 - acc: 0.5752 - val_loss: 0.6978 - val_acc: 0.5062
Epoch 19/60
11520/11520 [==============================] - 1s 71us/step - loss: 0.6792 - acc: 0.5751 - val_loss: 0.6983 - val_acc: 0.5062
Epoch 20/60
11520/11520 [==============================] - 1s 70us/step - loss: 0.6781 - acc: 0.5750 - val_loss: 0.7005 - val_acc: 0.5062
Epoch 21/60
11520/11520 [==============================] - 1s 73us/step - loss: 0.6786 - acc: 0.5749 - val_loss: 0.6975 - val_acc: 0.5062
Epoch 22/60
11520/11520 [==============================] - 1s 71us/step - loss: 0.6756 - acc: 0.5757 - val_loss: 0.6962 - val_acc: 0.5062
Epoch 23/60
11520/11520 [=====================

Epoch 15/60
11520/11520 [==============================] - 1s 67us/step - loss: 0.5008 - acc: 0.7803 - val_loss: 0.7571 - val_acc: 0.5945
Epoch 16/60
11520/11520 [==============================] - 1s 72us/step - loss: 0.4907 - acc: 0.7845 - val_loss: 0.6446 - val_acc: 0.6766
Epoch 17/60
11520/11520 [==============================] - 1s 72us/step - loss: 0.4885 - acc: 0.7882 - val_loss: 0.5328 - val_acc: 0.7320
Epoch 18/60
11520/11520 [==============================] - 1s 73us/step - loss: 0.4832 - acc: 0.7902 - val_loss: 0.5779 - val_acc: 0.7422
Epoch 19/60
11520/11520 [==============================] - 1s 72us/step - loss: 0.4859 - acc: 0.7878 - val_loss: 0.7213 - val_acc: 0.6406
Epoch 20/60
11520/11520 [==============================] - 1s 71us/step - loss: 0.4815 - acc: 0.7918 - val_loss: 0.5444 - val_acc: 0.7234
Epoch 21/60
11520/11520 [==============================] - 1s 73us/step - loss: 0.4755 - acc: 0.7943 - val_loss: 0.6278 - val_acc: 0.6742
Epoch 22/60
11520/11520 [=========

11520/11520 [==============================] - 1s 70us/step - loss: 0.5161 - acc: 0.7675 - val_loss: 0.6942 - val_acc: 0.6734
Epoch 14/60
11520/11520 [==============================] - 1s 70us/step - loss: 0.5093 - acc: 0.7735 - val_loss: 0.5352 - val_acc: 0.7367
Epoch 15/60
11520/11520 [==============================] - 1s 73us/step - loss: 0.4938 - acc: 0.7813 - val_loss: 0.9271 - val_acc: 0.5172
Epoch 16/60
11520/11520 [==============================] - 1s 70us/step - loss: 0.4938 - acc: 0.7826 - val_loss: 0.5693 - val_acc: 0.7344
Epoch 17/60
11520/11520 [==============================] - 1s 69us/step - loss: 0.4939 - acc: 0.7865 - val_loss: 0.5856 - val_acc: 0.7211
Epoch 18/60
11520/11520 [==============================] - 1s 72us/step - loss: 0.4770 - acc: 0.7929 - val_loss: 0.6925 - val_acc: 0.6711
Epoch 19/60
11520/11520 [==============================] - 1s 69us/step - loss: 0.4796 - acc: 0.7897 - val_loss: 0.5417 - val_acc: 0.7656
Epoch 20/60
11520/11520 [=====================

In [43]:
def SVM_linear(X_train, Y_train, X_test, C):
    gamma_range = np.logspace(-9, 3, 13)
    param_grid = dict(gamma=gamma_range)
    cv = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=42)
    grid = GridSearchCV(SVC(cache_size=2048), param_grid=param_grid, cv=cv, n_jobs=4)
    grid.fit(X_train, Y_train)

    print("The best parameters are %s with a score of %0.2f"
      % (grid.best_params_, grid.best_score_))

    clf = SVC(C=C, gamma=grid.best_params_['gamma'], class_weight='balanced', kernel='linear')
    clf.fit(X_train, Y_train)
    
    Y_pred = clf.predict(X_test)
    return Y_pred

def SVM_rbf(X_train, Y_train, X_test, C, gamma):

    clf = SVC(C=C, gamma=gamma, kernel='rbf')
    clf.fit(X_train, Y_train)
    
    Y_pred = clf.predict(X_test)
    return Y_pred

def SVM(array):
    array_train, array_val, array_test = SplitVectorData_Holdout(array, 0.6)
    X_train, Y_train, X_val, Y_val, X_test, Y_test = SplitLabelsFromPrimitives(array_train, array_val, array_test)
    
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_val = scaler.transform(X_val)
    X_test = scaler.transform(X_test)
    
    Y_pred_lin03= SVM_linear(X_train, Y_train, X_test, 1e-03)
    Y_pred_lin01= SVM_linear(X_train, Y_train, X_test, 1e-01)
    Y_pred_lin1= SVM_linear(X_train, Y_train, X_test, 1.0)
    Y_pred_lin10= SVM_linear(X_train, Y_train, X_test, 10.0)
    SVM_rbf(X_train, Y_train, X_test, 1e-03, 1e-03)
    SVM_rbf(X_train, Y_train, X_test, 1e-01, 1e-03)
    SVM_rbf(X_train, Y_train, X_test, 1.0, 1e-03)
    SVM_rbf(X_train, Y_train, X_test, 10.0, 1e-03)
    SVM_rbf(X_train, Y_train, X_test, 1e-03, 1e-01)
    SVM_rbf(X_train, Y_train, X_test, 1e-01, 1e-01)
    SVM_rbf(X_train, Y_train, X_test, 1.0, 1e-01)
    SVM_rbf(X_train, Y_train, X_test, 10.0, 1e-01)
    SVM_rbf(X_train, Y_train, X_test, 1e-03, 1.0)
    SVM_rbf(X_train, Y_train, X_test, 1e-01, 1.0)
    SVM_rbf(X_train, Y_train, X_test, 1.0, 1.0)
    SVM_rbf(X_train, Y_train, X_test, 10.0, 1.0)
    SVM_rbf(X_train, Y_train, X_test, 1e-03, 10.0)
    SVM_rbf(X_train, Y_train, X_test, 1e-01, 10.0)
    SVM_rbf(X_train, Y_train, X_test, 1.0, 10.0)
    SVM_rbf(X_train, Y_train, X_test, 10.0, 10.0)
    
    validations = [[Y_test, Y_pred_lin03],
    [Y_test, Y_pred_lin01],
    [Y_test, Y_pred_lin1],
    [Y_test, Y_pred_lin10],             
    [Y_test, Y_pred_rbf03_g03],
    [Y_test, Y_pred_rbf01_g03],
    [Y_test, Y_pred_rbf1_g03],
    [Y_test, Y_pred_rbf10_g03],            
    [Y_test, Y_pred_rbf03_g01],
    [Y_test, Y_pred_rbf01_g01],
    [Y_test, Y_pred_rbf1_g01],
    [Y_test, Y_pred_rbf10_g01],              
    [Y_test, Y_pred_rbf03_g1],
    [Y_test, Y_pred_rbf01_g1],
    [Y_test, Y_pred_rbf1_g1],
    [Y_test, Y_pred_rbf10_g1],            
    [Y_test, Y_pred_rbf03_g10],
    [Y_test, Y_pred_rbf01_g10],
    [Y_test, Y_pred_rbf1_g10],
    [Y_test, Y_pred_rbf10_g10]]
    
    return GenerateAccScores(validations)

    

In [42]:
accScores_svm = SVM(Galaxies)

The best parameters are {'gamma': 0.1} with a score of 0.84


AttributeError: 'GridSearchCV' object has no attribute 'best_params'